In [20]:
import pandas as pd
from nltk.tokenize import word_tokenize

In [2]:
from openie import StanfordOpenIE

In [3]:
import os
os.environ["CORENLP_HOME"] = r'D:\\Learning Material\\IR\\stanford-corenlp-full-2018-10-05\\stanford-corenlp-full-2018-10-05'

In [4]:
import string
table = str.maketrans('', '', string.punctuation)

In [5]:
#imports the data
roughDat= pd.read_csv("../../data/raw/Emergent_NAACL2016/emergent/url-versions-2015-06-14-clean.csv")

In [6]:
#removes unnecessary columns.
roughDat = roughDat.drop(columns=['Unnamed: 0', 'claimId', 'articleHeadlineStance'])

In [14]:
claims = roughDat["claimHeadline"].tolist()
headlines = roughDat["articleHeadline"].tolist()
claimSVO=[]
headSVO=[]

In [15]:
with StanfordOpenIE() as client:
    for i in claims:
        claimSVO.append(client.annotate(i))

Starting server with command: java -Xmx8G -cp C:\Users\adity\stanfordnlp_resources\stanford-corenlp-full-2018-10-05/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-ca25bb58592c4f3e.props -preload openie


In [16]:
with StanfordOpenIE() as client:
    for i in headlines:
        headSVO.append(client.annotate(i))

Starting server with command: java -Xmx8G -cp C:\Users\adity\stanfordnlp_resources\stanford-corenlp-full-2018-10-05/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-ae4d865c69934f8e.props -preload openie


In [17]:
claim_subjects = []
claim_objects = []
claim_relations = []
for claim_svo in claimSVO:
    t1 = set()
    t2 = set()
    t3 = set()
    for a in claim_svo:
        t1.add(a["subject"])
        t2.add(a["object"])
        t3.add(a["relation"])
    claim_subjects.append(t1)
    claim_objects.append(t2)
    claim_relations.append(t3)

In [18]:
headline_subjects = []
headline_objects = []
headline_relations = []
for headline_svo in headSVO:
    t1 = set()
    t2 = set()
    t3 = set()
    for a in headline_svo:
        t1.add(a["subject"])
        t2.add(a["object"])
        t3.add(a["relation"])
    headline_subjects.append(t1)
    headline_objects.append(t2)
    headline_relations.append(t3)

In [21]:
Equivalence=[]
for m,n in zip(claimSVO, headSVO):
    if len(m)!=0 and len(n)!=0:
        l2=word_tokenize(n[0]["subject"].lower())
        l1=word_tokenize(m[0]["subject"].lower())
        if len(l1)!=0 and len(l2)!=0:
            for i in l1:
                if i in l2:
                    Equivalence.append("Equivalence")
                    break
                else:
                    Equivalence.append("No Relation")

In [22]:
subject_equivalence = []
for cs, hs in zip(claim_subjects, headline_subjects):
    claim_tokens = set()
    headline_tokens = set()
    for sub in cs:
        claim_tokens.update(word_tokenize(sub.lower()))
    for sub in hs:
        headline_tokens.update(word_tokenize(sub.lower()))
    subject_equivalence.append(len(claim_tokens.intersection(headline_tokens)) == 0)

In [23]:
import pickle

In [26]:
# load the ppdb data
with open("../../data/external/ppdb-2.0-xl-lexical.pkl", "rb") as f:
    ppdb_dict = pickle.load(f)

In [27]:
subject_entailments = []
for cs, hs in zip(claim_subjects, headline_subjects):
    l = []
    entailment = ""
    for subject in cs:
        sub = subject.lower().translate(table)
        l.append(ppdb_dict.get(sub, {}))
    done = False
    for subject in hs:
        for para in l:
            entailment=para.get(subject,"noRelation")
            if entailment!="noRelation":
                subject_entailments.append(entailment[1])
                done = True
                break
        if done is True:
            break
    subject_entailments.append("noRelation")

In [28]:
relation_entailments = []
for cr, hr in zip(claim_relations, headline_relations):
    l = []
    entailment = ""
    for relation in cr:
        rel = relation.lower().translate(table)
        l.append(ppdb_dict.get(rel, {}))
    done = False
    for relation in hr:
        for para in l:
            entailment = para.get(relation,"noRelation")
            if entailment != "noRelation":
                relation_entailments.append(entailment[1])
                done = True
                break
        if done is True:
            break
    relation_entailments.append("noRelation")

In [29]:
object_entailments = []
for cr, hr in zip(claim_objects, headline_objects):
    l = []
    entailment = ""
    for obj in cr:
        rel = obj.lower().translate(table)
        l.append(ppdb_dict.get(rel, {}))
    done = False
    for obj in hr:
        for para in l:
            entailment = para.get(obj,"noRelation")
            if entailment != "noRelation":
                object_entailments.append(entailment[1])
                done = True
                break
        if done is True:
            break
    object_entailments.append("noRelation")

In [30]:
for i, (x,y) in enumerate(zip(subject_entailments, subject_equivalence)):
    if (x == "noRelation" or x == "Independent") and y is True:
        subject_entailments[i] = "Equivalence"

In [31]:
equiv_map = {
    "Independent": 0,
    "OtherRelated": 1,
    "Equivalence": 2,
    "ForwardEntailment": 3,
    "ReverseEntailment": 4,
    "Exclusion": 5,
    "noRelation": 0
}

In [32]:
object_entailments = [equiv_map[i] for i in object_entailments]
relation_entailments = [equiv_map[i] for i in relation_entailments]
subject_entailments = [equiv_map[i] for i in subject_entailments]

In [34]:
import numpy as np

In [38]:
df = pd.DataFrame({
    "articleId": roughDat["articleId"],
    "subject_entailments": subject_entailments,
    "object_entailments": object_entailments,
    "relation_entailments": relation_entailments
})

In [39]:
df.to_csv("../../data/processed/features/svo.csv", index=False)